In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings(action='ignore')
import pickle
import time

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
#from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import sklearn.svm as svm
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from matplotlib_venn import venn2, venn2_circles

from xgboost import plot_importance
from scipy.stats import randint
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
def seed_everything(seed):
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed) #넘파이를 사용할 경우
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    
seed_everything(37) # Seed 고정

In [ ]:
!jupyter nbconvert --clear-output --inplace 3_ModelTraining_xgboost_grid_결측치처리TO제품만knn5.ipynb

In [ ]:
# # Preprocessing 전처리한 데이터 train_pp, test_pp
# train_a = pd.read_csv('./data/train_a.csv')
# train_to = pd.read_csv('./data/train_to.csv')
# traina_pp = pd.read_csv('./data/imputed_a_1val_highcorr.csv')
# trainto_pp = pd.read_csv('./data/imputed_to_1val_highcorr.csv')

# test = pd.read_csv('./data/test.csv')
# test_a = test[test['PRODUCT_CODE']=='A_31']
# test_to = test[(test['PRODUCT_CODE']=='T_31')|(test['PRODUCT_CODE']=='O_31')]

In [3]:
a_df = pd.read_csv('./data/a_df.csv')
to_df = pd.read_csv('./data/imp_to_df.csv')

test = pd.read_csv('./data/test.csv')
test_a = test[test['PRODUCT_CODE']=='A_31']
test_to = test[(test['PRODUCT_CODE']=='T_31')|(test['PRODUCT_CODE']=='O_31')]

In [4]:
display(a_df.head())
display(to_df.head())
display(test_a.head())
display(test_to.head())

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_129,X_130,X_132,X_133,...,X_2770,X_2771,X_2772,X_2773,X_2775,X_2776,X_2837,X_2839,X_2840,X_2871
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,7813.0,NaN,0.19,0.20,...,230.0,229.974074,229.9,NaN,NaN,290.0,9.04,5.02,7.00,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,19854.0,0.20,0.21,...,230.0,229.980000,229.9,0.0,0.0,290.0,8.00,5.00,7.05,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,7815.0,NaN,0.19,0.20,...,230.1,229.992593,229.9,NaN,NaN,290.0,9.04,5.02,6.52,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,19856.0,0.20,0.21,...,230.1,230.018518,229.9,0.0,0.0,290.0,7.49,5.01,7.03,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,7817.0,NaN,0.19,0.20,...,230.1,230.007407,229.9,NaN,NaN,291.0,9.04,5.03,6.52,NaN


,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_5,X_7,...,X_905,X_906,X_907,X_908,X_909,X_910,X_912,X_931,X_932,X_933
0,TRAIN_022,0,0.517719,2022-06-14 8:53,T100304,T_31,2.0,102.0,11.0,45.0,...,0.0,17.28,17.277333,17.27,0.01,0.7,0.6,13.70,13.443333,13.20
1,TRAIN_023,0,0.519090,2022-06-14 9:01,T100304,T_31,2.0,102.0,11.0,45.0,...,0.0,17.28,17.276886,17.27,0.01,0.7,0.6,13.68,13.438344,13.22
2,TRAIN_025,1,0.529362,2022-06-19 9:11,T100304,T_31,2.0,97.0,11.0,45.0,...,1.0,17.28,17.276452,17.27,0.01,0.7,0.6,13.70,13.454839,13.20
3,TRAIN_026,1,0.531992,2022-06-19 9:20,T100306,T_31,2.0,95.0,10.0,54.0,...,1.0,17.28,17.276452,17.27,0.01,0.7,0.6,13.70,13.461290,13.30
4,TRAIN_029,1,0.532405,2022-06-19 23:31,T100304,T_31,2.0,100.0,11.0,45.0,...,1.0,17.28,17.277419,17.27,0.01,0.7,0.6,13.70,13.422581,13.20


,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TEST_005,2022-09-09 19:35,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TEST_006,2022-09-09 19:43,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,TEST_007,2022-09-10 12:27,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,57.74,52.51,54.45,57.99,63.16,1.0,NaN,NaN,NaN,NaN


,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,TEST_015,2022-09-20 5:20,T100304,T_31,2.0,102.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,TEST_016,2022-09-20 5:28,T100306,T_31,2.0,93.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 데이터 준비

train데이터로 학습한 모델로 test데이터의 예측을 진행하므로 train의 컬럼만 test데이터에 넣는다.

In [5]:
testa_df = test_a[a_df.iloc[:,6:].columns]
testto_df = test_to[to_df.iloc[:,6:].columns]
display(testa_df.head())
display(testto_df.head())

,X_129,X_130,X_132,X_133,X_134,X_135,X_141,X_142,X_143,X_144,...,X_2770,X_2771,X_2772,X_2773,X_2775,X_2776,X_2837,X_2839,X_2840,X_2871
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,18031.0,NaN,0.19,0.2,0.19,0.2,575.6,0.0,0.0,249.96,...,230.0,229.971429,229.8,NaN,NaN,290.0,9.04,5.02,7.02,1.0


,X_1,X_2,X_5,X_7,X_8,X_9,X_11,X_15,X_16,X_17,...,X_905,X_906,X_907,X_908,X_909,X_910,X_912,X_931,X_932,X_933
0,2.0,94.0,10.0,51.0,10.0,52.0,469.6,0.0,247.5,248.0,...,0.0,17.28,17.278387,17.27,0.01,0.7,0.6,13.6,13.383871,13.2
1,2.0,93.0,11.0,45.0,10.0,31.0,506.6,0.0,247.6,248.2,...,0.0,17.28,17.277742,17.27,0.01,0.7,0.6,13.6,13.406452,13.2
2,2.0,95.0,11.0,45.0,10.0,31.0,506.6,0.0,247.6,247.9,...,0.0,17.28,17.277097,17.27,0.01,0.7,0.6,13.6,13.370968,13.2
15,2.0,102.0,11.0,45.0,10.0,31.0,520.3,0.0,247.5,248.1,...,0.0,17.28,17.276774,17.27,0.01,0.7,0.6,13.6,13.387097,13.2
16,2.0,93.0,10.0,54.0,10.0,52.0,472.5,0.0,247.5,248.0,...,0.0,17.28,17.275806,17.27,0.01,0.7,0.6,13.6,13.367742,13.2


In [6]:
train_y = a_df['Y_Class']

In [7]:
# train_x = pd.concat([a_df[['LINE', 'PRODUCT_CODE']],traina_pp], axis = 1)
# train_x.head()

In [8]:
train_x = a_df.drop(columns = ['PRODUCT_ID','TIMESTAMP','Y_Class', 'Y_Quality'])

In [9]:
test_x = pd.concat([test_a[['LINE', 'PRODUCT_CODE']],testa_df], axis = 1)
test_x.head()

,LINE,PRODUCT_CODE,X_129,X_130,X_132,X_133,X_134,X_135,X_141,X_142,...,X_2770,X_2771,X_2772,X_2773,X_2775,X_2776,X_2837,X_2839,X_2840,X_2871
3,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,T050304,A_31,18031.0,NaN,0.19,0.2,0.19,0.2,575.6,0.0,...,230.0,229.971429,229.8,NaN,NaN,290.0,9.04,5.02,7.02,1.0


In [10]:
a_index = test_x.index.tolist() 

In [11]:
len(a_index)

67

### A 데이터 범주형 변수 라벨 인코딩

In [12]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i]) #원래 column 값을 기준으로 fit.
    train_x[i] = le.transform(train_x[i]) #수치화, 수치로 변형

    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


## train_a 학습 및 test_a 예측

In [21]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb_model = XGBClassifier(n_estimators=1500)

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, 
                                                    test_size=0.3, 
                                                    stratify=train_y, 
                                                    random_state=37)

# 후보 파라미터 선정
params = {'max_depth':[6], #일반적으로 3~10 6
          'subsample':[0.8], #일반적으로 0.5 ~ 1
          'learning_rate':[0.12], #일반적으로 0.01 ~ 0.2 0.12
          'min_child_weight':[4, 5], # 1보다 샘플수작으면 leaf node. 너무 크게 설정하면 언더피팅 4
          'reg_lambda': [1], #default 1, L2 규제
          'reg_alpha' : [0], #default 0, L1 규제  
          'min_split_loss':[0.02,0.03], #범위 0~무한대 0.02
          'colsample_bytree':[0.8]} #일반적으로 0~1 0.8

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 시작 / eval_set: 학습은  (X_train, y_train), 검증은 (X_test, y_test)
gridcv.fit(X_train, y_train, early_stopping_rounds=50, eval_metric='mlogloss', eval_set=[(X_train, y_train), (X_test, y_test)])

#튜닝된 파라미터 출력
print(gridcv.best_params_)
print(gridcv.best_score_)

best_params = gridcv.best_params_
best_xgb_model = XGBClassifier(n_estimators=1500, **best_params)

# 훈련 데이터 전체에 대해 모델 학습
best_xgb_model.fit(train_x, train_y)

# 새로운 데이터에 대해 예측
preds = best_xgb_model.predict(test_x)

[0]	validation_0-mlogloss:1.03785	validation_1-mlogloss:1.06746
[1]	validation_0-mlogloss:0.98127	validation_1-mlogloss:1.03210
[2]	validation_0-mlogloss:0.92931	validation_1-mlogloss:1.01349
[3]	validation_0-mlogloss:0.88577	validation_1-mlogloss:0.99253
[4]	validation_0-mlogloss:0.83971	validation_1-mlogloss:0.97069
[5]	validation_0-mlogloss:0.80218	validation_1-mlogloss:0.96547
[6]	validation_0-mlogloss:0.77196	validation_1-mlogloss:0.94994
[7]	validation_0-mlogloss:0.74471	validation_1-mlogloss:0.95177
[8]	validation_0-mlogloss:0.71499	validation_1-mlogloss:0.94339
[9]	validation_0-mlogloss:0.69014	validation_1-mlogloss:0.94320
[10]	validation_0-mlogloss:0.66548	validation_1-mlogloss:0.93527
[11]	validation_0-mlogloss:0.64636	validation_1-mlogloss:0.93358
[12]	validation_0-mlogloss:0.62600	validation_1-mlogloss:0.93579
[13]	validation_0-mlogloss:0.60975	validation_1-mlogloss:0.93775
[14]	validation_0-mlogloss:0.59466	validation_1-mlogloss:0.93143
[15]	validation_0-mlogloss:0.58022	

[56]	validation_0-mlogloss:0.40724	validation_1-mlogloss:1.06809
[57]	validation_0-mlogloss:0.40504	validation_1-mlogloss:1.07068
[58]	validation_0-mlogloss:0.40466	validation_1-mlogloss:1.07307
[59]	validation_0-mlogloss:0.40412	validation_1-mlogloss:1.07464
[60]	validation_0-mlogloss:0.40264	validation_1-mlogloss:1.07988
[0]	validation_0-mlogloss:1.03924	validation_1-mlogloss:1.07492
[1]	validation_0-mlogloss:0.99172	validation_1-mlogloss:1.05697
[2]	validation_0-mlogloss:0.94493	validation_1-mlogloss:1.03248
[3]	validation_0-mlogloss:0.90405	validation_1-mlogloss:1.01157
[4]	validation_0-mlogloss:0.86273	validation_1-mlogloss:0.98712
[5]	validation_0-mlogloss:0.82721	validation_1-mlogloss:0.97393
[6]	validation_0-mlogloss:0.80212	validation_1-mlogloss:0.96592
[7]	validation_0-mlogloss:0.77418	validation_1-mlogloss:0.96147
[8]	validation_0-mlogloss:0.74716	validation_1-mlogloss:0.95133
[9]	validation_0-mlogloss:0.72104	validation_1-mlogloss:0.94399
[10]	validation_0-mlogloss:0.69855	

[49]	validation_0-mlogloss:0.35934	validation_1-mlogloss:0.95417
[50]	validation_0-mlogloss:0.35664	validation_1-mlogloss:0.95716
[51]	validation_0-mlogloss:0.35335	validation_1-mlogloss:0.95950
[52]	validation_0-mlogloss:0.35187	validation_1-mlogloss:0.96237
[53]	validation_0-mlogloss:0.35097	validation_1-mlogloss:0.96233
[54]	validation_0-mlogloss:0.34814	validation_1-mlogloss:0.96160
[55]	validation_0-mlogloss:0.34568	validation_1-mlogloss:0.96174
[56]	validation_0-mlogloss:0.34450	validation_1-mlogloss:0.96270
[57]	validation_0-mlogloss:0.34303	validation_1-mlogloss:0.96448
[58]	validation_0-mlogloss:0.34190	validation_1-mlogloss:0.96594
[59]	validation_0-mlogloss:0.34124	validation_1-mlogloss:0.97032
[60]	validation_0-mlogloss:0.34101	validation_1-mlogloss:0.97342
[61]	validation_0-mlogloss:0.33921	validation_1-mlogloss:0.97535
[62]	validation_0-mlogloss:0.33761	validation_1-mlogloss:0.97764
[63]	validation_0-mlogloss:0.33462	validation_1-mlogloss:0.97480
[64]	validation_0-mloglos

[43]	validation_0-mlogloss:0.43223	validation_1-mlogloss:0.98283
[44]	validation_0-mlogloss:0.42947	validation_1-mlogloss:0.98796
[45]	validation_0-mlogloss:0.42694	validation_1-mlogloss:0.99155
[46]	validation_0-mlogloss:0.42563	validation_1-mlogloss:0.99620
[47]	validation_0-mlogloss:0.42350	validation_1-mlogloss:1.00272
[48]	validation_0-mlogloss:0.42121	validation_1-mlogloss:1.00883
[49]	validation_0-mlogloss:0.41814	validation_1-mlogloss:1.00942
[50]	validation_0-mlogloss:0.41641	validation_1-mlogloss:1.01116
[51]	validation_0-mlogloss:0.41476	validation_1-mlogloss:1.01623
[52]	validation_0-mlogloss:0.41340	validation_1-mlogloss:1.01902
[53]	validation_0-mlogloss:0.41367	validation_1-mlogloss:1.02188
[54]	validation_0-mlogloss:0.41282	validation_1-mlogloss:1.02633
[55]	validation_0-mlogloss:0.41267	validation_1-mlogloss:1.03041
[56]	validation_0-mlogloss:0.41137	validation_1-mlogloss:1.03252
[57]	validation_0-mlogloss:0.40992	validation_1-mlogloss:1.03805
[58]	validation_0-mloglos

[19]	validation_0-mlogloss:0.57691	validation_1-mlogloss:0.95922
[20]	validation_0-mlogloss:0.56737	validation_1-mlogloss:0.96072
[21]	validation_0-mlogloss:0.55825	validation_1-mlogloss:0.95920
[22]	validation_0-mlogloss:0.54951	validation_1-mlogloss:0.96251
[23]	validation_0-mlogloss:0.54305	validation_1-mlogloss:0.96107
[24]	validation_0-mlogloss:0.53625	validation_1-mlogloss:0.96546
[25]	validation_0-mlogloss:0.53085	validation_1-mlogloss:0.97489
[26]	validation_0-mlogloss:0.52245	validation_1-mlogloss:0.98754
[27]	validation_0-mlogloss:0.51498	validation_1-mlogloss:0.98758
[28]	validation_0-mlogloss:0.50957	validation_1-mlogloss:0.98501
[29]	validation_0-mlogloss:0.50434	validation_1-mlogloss:0.98886
[30]	validation_0-mlogloss:0.49800	validation_1-mlogloss:0.99515
[31]	validation_0-mlogloss:0.49521	validation_1-mlogloss:0.99878
[32]	validation_0-mlogloss:0.49016	validation_1-mlogloss:1.00114
[33]	validation_0-mlogloss:0.48720	validation_1-mlogloss:1.00121
[34]	validation_0-mloglos

...

[74]	validation_0-mlogloss:0.32695	validation_1-mlogloss:0.99850
[75]	validation_0-mlogloss:0.32670	validation_1-mlogloss:0.99770
[76]	validation_0-mlogloss:0.32644	validation_1-mlogloss:0.99818
[77]	validation_0-mlogloss:0.32637	validation_1-mlogloss:0.99871
[0]	validation_0-mlogloss:1.03260	validation_1-mlogloss:1.07004
[1]	validation_0-mlogloss:0.97747	validation_1-mlogloss:1.04172
[2]	validation_0-mlogloss:0.93009	validation_1-mlogloss:1.01939
[3]	validation_0-mlogloss:0.88520	validation_1-mlogloss:0.99957
[4]	validation_0-mlogloss:0.83800	validation_1-mlogloss:0.98005
[5]	validation_0-mlogloss:0.80254	validation_1-mlogloss:0.97792
[6]	validation_0-mlogloss:0.76633	validation_1-mlogloss:0.96854
[7]	validation_0-mlogloss:0.74071	validation_1-mlogloss:0.96000
[8]	validation_0-mlogloss:0.71299	validation_1-mlogloss:0.95874
[9]	validation_0-mlogloss:0.69418	validation_1-mlogloss:0.96162
[10]	validation_0-mlogloss:0.66974	validation_1-mlogloss:0.95921
[11]	validation_0-mlogloss:0.65610	

[61]	validation_0-mlogloss:0.41543	validation_1-mlogloss:1.04215
[0]	validation_0-mlogloss:1.03957	validation_1-mlogloss:1.06607
[1]	validation_0-mlogloss:0.99062	validation_1-mlogloss:1.03221
[2]	validation_0-mlogloss:0.95005	validation_1-mlogloss:1.02369
[3]	validation_0-mlogloss:0.91100	validation_1-mlogloss:1.00574
[4]	validation_0-mlogloss:0.86772	validation_1-mlogloss:0.98614
[5]	validation_0-mlogloss:0.83675	validation_1-mlogloss:0.97640
[6]	validation_0-mlogloss:0.81025	validation_1-mlogloss:0.96095
[7]	validation_0-mlogloss:0.78408	validation_1-mlogloss:0.95675
[8]	validation_0-mlogloss:0.75456	validation_1-mlogloss:0.95406
[9]	validation_0-mlogloss:0.72788	validation_1-mlogloss:0.94920
[10]	validation_0-mlogloss:0.70370	validation_1-mlogloss:0.93423
[11]	validation_0-mlogloss:0.68309	validation_1-mlogloss:0.92960
[12]	validation_0-mlogloss:0.66131	validation_1-mlogloss:0.92736
[13]	validation_0-mlogloss:0.64333	validation_1-mlogloss:0.92048
[14]	validation_0-mlogloss:0.62722	

[47]	validation_0-mlogloss:0.44799	validation_1-mlogloss:1.03767
[48]	validation_0-mlogloss:0.44824	validation_1-mlogloss:1.04079
[49]	validation_0-mlogloss:0.44755	validation_1-mlogloss:1.04533
[50]	validation_0-mlogloss:0.44647	validation_1-mlogloss:1.05044
[51]	validation_0-mlogloss:0.44554	validation_1-mlogloss:1.05030
[52]	validation_0-mlogloss:0.44425	validation_1-mlogloss:1.05378
[53]	validation_0-mlogloss:0.44268	validation_1-mlogloss:1.05326
[54]	validation_0-mlogloss:0.44169	validation_1-mlogloss:1.05536
[55]	validation_0-mlogloss:0.43904	validation_1-mlogloss:1.06356
[56]	validation_0-mlogloss:0.43735	validation_1-mlogloss:1.06562
[57]	validation_0-mlogloss:0.43549	validation_1-mlogloss:1.06914
[58]	validation_0-mlogloss:0.43492	validation_1-mlogloss:1.07175
[59]	validation_0-mlogloss:0.43356	validation_1-mlogloss:1.07116
[60]	validation_0-mlogloss:0.43217	validation_1-mlogloss:1.06959
[61]	validation_0-mlogloss:0.43202	validation_1-mlogloss:1.07041
[62]	validation_0-mloglos

[48]	validation_0-mlogloss:0.40201	validation_1-mlogloss:0.90929
[49]	validation_0-mlogloss:0.40032	validation_1-mlogloss:0.91103
[50]	validation_0-mlogloss:0.39757	validation_1-mlogloss:0.91713
[51]	validation_0-mlogloss:0.39359	validation_1-mlogloss:0.91838
[52]	validation_0-mlogloss:0.39293	validation_1-mlogloss:0.92342
[53]	validation_0-mlogloss:0.39177	validation_1-mlogloss:0.92599
[54]	validation_0-mlogloss:0.38838	validation_1-mlogloss:0.92184
[55]	validation_0-mlogloss:0.38652	validation_1-mlogloss:0.92108
[56]	validation_0-mlogloss:0.38520	validation_1-mlogloss:0.92182
[57]	validation_0-mlogloss:0.38382	validation_1-mlogloss:0.92617
[58]	validation_0-mlogloss:0.38294	validation_1-mlogloss:0.92693
[59]	validation_0-mlogloss:0.38203	validation_1-mlogloss:0.93123
[60]	validation_0-mlogloss:0.38172	validation_1-mlogloss:0.92809
[61]	validation_0-mlogloss:0.38027	validation_1-mlogloss:0.92875
[62]	validation_0-mlogloss:0.37858	validation_1-mlogloss:0.93158
[63]	validation_0-mloglos

[34]	validation_0-mlogloss:0.50351	validation_1-mlogloss:0.98585
[35]	validation_0-mlogloss:0.49701	validation_1-mlogloss:0.98684
[36]	validation_0-mlogloss:0.49298	validation_1-mlogloss:0.98965
[37]	validation_0-mlogloss:0.48953	validation_1-mlogloss:0.99382
[38]	validation_0-mlogloss:0.48740	validation_1-mlogloss:0.99428
[39]	validation_0-mlogloss:0.48137	validation_1-mlogloss:0.99904
[40]	validation_0-mlogloss:0.47881	validation_1-mlogloss:0.99749
[41]	validation_0-mlogloss:0.47732	validation_1-mlogloss:1.00076
[42]	validation_0-mlogloss:0.47384	validation_1-mlogloss:0.99742
[43]	validation_0-mlogloss:0.47158	validation_1-mlogloss:0.99927
[44]	validation_0-mlogloss:0.46864	validation_1-mlogloss:1.00483
[45]	validation_0-mlogloss:0.46586	validation_1-mlogloss:1.00355
[46]	validation_0-mlogloss:0.46299	validation_1-mlogloss:1.00541
[47]	validation_0-mlogloss:0.46122	validation_1-mlogloss:1.00811
[48]	validation_0-mlogloss:0.46047	validation_1-mlogloss:1.00884
[49]	validation_0-mloglos

In [22]:
preds = pd.DataFrame(preds)
preds.index = a_index
preds

,0
3,1
4,2
5,2
6,1
7,0
...,...
284,0
285,0
286,0
292,0


In [23]:
preds.describe()

,0
count,67.000000
mean,0.656716
std,0.769735
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,2.000000


# 데이터 준비

train데이터로 학습한 모델로 test데이터의 예측을 진행하므로 train의 컬럼만 test데이터에 넣는다.

train과 test 데이터는 필요없는 컬럼들 제거하고 knn 보간법으로 결측치 처리된 데이터

In [24]:
# testa_df = test_a[a_df.iloc[:,6:].columns]
# testto_df = test_to[to_df.iloc[:,6:].columns]

In [25]:
train_y = to_df['Y_Class']

In [26]:
train_x = to_df.drop(columns = ['PRODUCT_ID','TIMESTAMP','Y_Class', 'Y_Quality'])

In [27]:
test_x = pd.concat([test_to[['LINE', 'PRODUCT_CODE']],testto_df], axis = 1)
test_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_5,X_7,X_8,X_9,X_11,X_15,...,X_905,X_906,X_907,X_908,X_909,X_910,X_912,X_931,X_932,X_933
0,T100306,T_31,2.0,94.0,10.0,51.0,10.0,52.0,469.6,0.0,...,0.0,17.28,17.278387,17.27,0.01,0.7,0.6,13.6,13.383871,13.2
1,T100304,T_31,2.0,93.0,11.0,45.0,10.0,31.0,506.6,0.0,...,0.0,17.28,17.277742,17.27,0.01,0.7,0.6,13.6,13.406452,13.2
2,T100304,T_31,2.0,95.0,11.0,45.0,10.0,31.0,506.6,0.0,...,0.0,17.28,17.277097,17.27,0.01,0.7,0.6,13.6,13.370968,13.2
15,T100304,T_31,2.0,102.0,11.0,45.0,10.0,31.0,520.3,0.0,...,0.0,17.28,17.276774,17.27,0.01,0.7,0.6,13.6,13.387097,13.2
16,T100306,T_31,2.0,93.0,10.0,54.0,10.0,52.0,472.5,0.0,...,0.0,17.28,17.275806,17.27,0.01,0.7,0.6,13.6,13.367742,13.2


In [28]:
to_index = test_x.index.tolist() 

### TO 데이터 범주형 변수 라벨 인코딩

In [29]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i]) #원래 column 값을 기준으로 fit.
    train_x[i] = le.transform(train_x[i]) #수치화, 수치로 변형

    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


## train_to 학습 및 test_to 예측

In [37]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb_model = XGBClassifier(n_estimators=1500)

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, 
                                                    test_size=0.3, 
                                                    stratify=train_y, 
                                                    random_state=37)

# 후보 파라미터 선정
params = {'max_depth':[5], #일반적으로 3~10
          'subsample':[0.92], #일반적으로 0.5 ~ 1
          'learning_rate':[0.1], #일반적으로 0.01 ~ 0.2
          'min_child_weight':[2], # 1보다 샘플수작으면 leaf node. 너무 크게 설정하면 언더피팅
          'reg_lambda': [1], #default 1, L2 규제
          'reg_alpha' : [0], #default 0, L1 규제  
          'min_split_loss':[0.12], #범위 0~무한대
          'colsample_bytree':[0.88]} #일반적으로 0~1

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 시작 / eval_set: 학습은  (X_train, y_train), 검증은 (X_test, y_test)
gridcv.fit(X_train, y_train, early_stopping_rounds=50, eval_metric='mlogloss', eval_set=[(X_train, y_train), (X_test, y_test)])

#튜닝된 파라미터 출력
print(gridcv.best_params_)
print(gridcv.best_score_)

best_params = gridcv.best_params_
best_xgb_model = XGBClassifier(n_estimators=1500, **best_params)

# 훈련 데이터 전체에 대해 모델 학습
best_xgb_model.fit(train_x, train_y)

# 새로운 데이터에 대해 예측
preds2 = best_xgb_model.predict(test_x)

[0]	validation_0-mlogloss:0.99932	validation_1-mlogloss:1.01561
[1]	validation_0-mlogloss:0.91682	validation_1-mlogloss:0.95391
[2]	validation_0-mlogloss:0.84433	validation_1-mlogloss:0.90434
[3]	validation_0-mlogloss:0.78126	validation_1-mlogloss:0.85513
[4]	validation_0-mlogloss:0.72278	validation_1-mlogloss:0.81751
[5]	validation_0-mlogloss:0.67328	validation_1-mlogloss:0.78439
[6]	validation_0-mlogloss:0.63102	validation_1-mlogloss:0.75285
[7]	validation_0-mlogloss:0.59296	validation_1-mlogloss:0.72291
[8]	validation_0-mlogloss:0.56055	validation_1-mlogloss:0.70217
[9]	validation_0-mlogloss:0.52814	validation_1-mlogloss:0.67813
[10]	validation_0-mlogloss:0.50152	validation_1-mlogloss:0.66477
[11]	validation_0-mlogloss:0.47506	validation_1-mlogloss:0.65116
[12]	validation_0-mlogloss:0.45175	validation_1-mlogloss:0.64152
[13]	validation_0-mlogloss:0.42860	validation_1-mlogloss:0.62790
[14]	validation_0-mlogloss:0.41046	validation_1-mlogloss:0.61993
[15]	validation_0-mlogloss:0.39479	

[51]	validation_0-mlogloss:0.24308	validation_1-mlogloss:0.61742
[52]	validation_0-mlogloss:0.24239	validation_1-mlogloss:0.62183
[53]	validation_0-mlogloss:0.24249	validation_1-mlogloss:0.62197
[54]	validation_0-mlogloss:0.24264	validation_1-mlogloss:0.62254
[55]	validation_0-mlogloss:0.24291	validation_1-mlogloss:0.62465
[56]	validation_0-mlogloss:0.24362	validation_1-mlogloss:0.62788
[57]	validation_0-mlogloss:0.24403	validation_1-mlogloss:0.63291
[58]	validation_0-mlogloss:0.24433	validation_1-mlogloss:0.63420
[59]	validation_0-mlogloss:0.24405	validation_1-mlogloss:0.63542
[60]	validation_0-mlogloss:0.24409	validation_1-mlogloss:0.63916
[61]	validation_0-mlogloss:0.24444	validation_1-mlogloss:0.64294
[62]	validation_0-mlogloss:0.24444	validation_1-mlogloss:0.64432
[63]	validation_0-mlogloss:0.24474	validation_1-mlogloss:0.64838
[64]	validation_0-mlogloss:0.24497	validation_1-mlogloss:0.65048
[65]	validation_0-mlogloss:0.24564	validation_1-mlogloss:0.65178
[66]	validation_0-mloglos

[29]	validation_0-mlogloss:0.26450	validation_1-mlogloss:0.60010
[30]	validation_0-mlogloss:0.26077	validation_1-mlogloss:0.60417
[31]	validation_0-mlogloss:0.25764	validation_1-mlogloss:0.60412
[32]	validation_0-mlogloss:0.25422	validation_1-mlogloss:0.60703
[33]	validation_0-mlogloss:0.25128	validation_1-mlogloss:0.60795
[34]	validation_0-mlogloss:0.24968	validation_1-mlogloss:0.61203
[35]	validation_0-mlogloss:0.24667	validation_1-mlogloss:0.61617
[36]	validation_0-mlogloss:0.24483	validation_1-mlogloss:0.62074
[37]	validation_0-mlogloss:0.24359	validation_1-mlogloss:0.62237
[38]	validation_0-mlogloss:0.24240	validation_1-mlogloss:0.62553
[39]	validation_0-mlogloss:0.24063	validation_1-mlogloss:0.62660
[40]	validation_0-mlogloss:0.23809	validation_1-mlogloss:0.62772
[41]	validation_0-mlogloss:0.23631	validation_1-mlogloss:0.62939
[42]	validation_0-mlogloss:0.23595	validation_1-mlogloss:0.63380
[43]	validation_0-mlogloss:0.23557	validation_1-mlogloss:0.63554
[44]	validation_0-mloglos

[6]	validation_0-mlogloss:0.63677	validation_1-mlogloss:0.74416
[7]	validation_0-mlogloss:0.59751	validation_1-mlogloss:0.71745
[8]	validation_0-mlogloss:0.56267	validation_1-mlogloss:0.69560
[9]	validation_0-mlogloss:0.53102	validation_1-mlogloss:0.67617
[10]	validation_0-mlogloss:0.50490	validation_1-mlogloss:0.65816
[11]	validation_0-mlogloss:0.47912	validation_1-mlogloss:0.64098
[12]	validation_0-mlogloss:0.45674	validation_1-mlogloss:0.63062
[13]	validation_0-mlogloss:0.43524	validation_1-mlogloss:0.61735
[14]	validation_0-mlogloss:0.41626	validation_1-mlogloss:0.61168
[15]	validation_0-mlogloss:0.39898	validation_1-mlogloss:0.60240
[16]	validation_0-mlogloss:0.38354	validation_1-mlogloss:0.59245
[17]	validation_0-mlogloss:0.37131	validation_1-mlogloss:0.58826
[18]	validation_0-mlogloss:0.35951	validation_1-mlogloss:0.58168
[19]	validation_0-mlogloss:0.34837	validation_1-mlogloss:0.58013
[20]	validation_0-mlogloss:0.33850	validation_1-mlogloss:0.57705
[21]	validation_0-mlogloss:0.

[57]	validation_0-mlogloss:0.05292	validation_1-mlogloss:0.64380
[58]	validation_0-mlogloss:0.05188	validation_1-mlogloss:0.64687
[59]	validation_0-mlogloss:0.05075	validation_1-mlogloss:0.64953
[60]	validation_0-mlogloss:0.04980	validation_1-mlogloss:0.65249
[61]	validation_0-mlogloss:0.04886	validation_1-mlogloss:0.65418
[62]	validation_0-mlogloss:0.04773	validation_1-mlogloss:0.65600
[63]	validation_0-mlogloss:0.04671	validation_1-mlogloss:0.65826
[64]	validation_0-mlogloss:0.04582	validation_1-mlogloss:0.66068
[65]	validation_0-mlogloss:0.04503	validation_1-mlogloss:0.66314
[66]	validation_0-mlogloss:0.04429	validation_1-mlogloss:0.66409
[67]	validation_0-mlogloss:0.04354	validation_1-mlogloss:0.66728
[68]	validation_0-mlogloss:0.04270	validation_1-mlogloss:0.66948
[69]	validation_0-mlogloss:0.04195	validation_1-mlogloss:0.67246
[70]	validation_0-mlogloss:0.04122	validation_1-mlogloss:0.67541
[71]	validation_0-mlogloss:0.04050	validation_1-mlogloss:0.67609
{'colsample_bytree': 0.88

In [38]:
preds2 = pd.DataFrame(preds2)
preds2.index = to_index
preds2

,0
0,1
1,1
2,1
15,1
16,1
...,...
305,1
306,1
307,1
308,1


In [39]:
preds2.describe()

,0
count,243.000000
mean,1.008230
std,0.090534
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [40]:
prediction = pd.concat([preds, preds2], axis = 0)
prediction = prediction.sort_index(ascending=True)
prediction

,0
0,1
1,1
2,1
3,1
4,2
...,...
305,1
306,1
307,1
308,1


In [41]:
prediction.describe()

,0
count,310.000000
mean,0.932258
std,0.392393
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [42]:
sub = pd.read_csv('./sample_submission.csv')
sub['Y_Class'] = prediction
#이상치처리x, 결측치knn보간법, std0제거, 높은 상관성 제거,  to제품 결측치처리
sub.to_csv('./data/sub/sub_grid7.csv', index = False)

In [ ]:
!jupyter nbconvert --clear-output --inplace 3_ModelTraining_xgboost_grid.ipynb